In [2]:
import yfinance as yf
from datetime import datetime
import warnings

# Suprimir avisos específicos
warnings.filterwarnings("ignore", category=FutureWarning, module="yfinance")


## Yahoo Fundamental

In [38]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
def get_yahoo_ticker_holders(ticker: str) -> pd.DataFrame:
    """
    Extrat data from yahoo holders for ticker.

    Parameters:
        classe_tabela (str): Class CSS from table to scrap (optional).

    Returns:
        pd.DataFrame: DataFrame with table content.
    """

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get(f"https://finance.yahoo.com/quote/{ticker}/holders/", headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        tabela = soup.find('section', {'data-testid': "holders-top-institutional-holders"})

        headers = [th.text.strip() for th in tabela.find_all('th')]
        
        rows = []
        for row in tabela.find_all('tr')[1:]:
            cols = [td.text.strip() for td in row.find_all('td')]
            if cols: 
                rows.append(cols)

        df = pd.DataFrame(rows, columns=headers if headers else None)
        df["% Out"] = df["% Out"].str.replace("%", "", regex=False)
        
    except requests.exceptions.RequestException as e:
        print(f"Error accessing URL: {e}")
    except Exception as e:
        print(f"Error processing data: {e}")

    return df


In [ ]:
data = get_yahoo_ticker_holders(ticker="AAPL")
data

,Holder,Shares,Date Reported,% Out,Value
0,Vanguard Group Inc,1.4B,"Dec 31, 2024",9.29,"342,735,136,731"
1,Blackrock Inc.,1.12B,"Dec 31, 2024",7.48,"275,855,196,827"
2,State Street Corporation,595.5M,"Dec 31, 2024",3.96,"146,225,396,861"
3,"FMR, LLC",341.64M,"Dec 31, 2024",2.27,"83,889,942,208"
4,"Geode Capital Management, LLC",340.16M,"Dec 31, 2024",2.26,"83,527,495,425"
5,"Berkshire Hathaway, Inc",300M,"Dec 31, 2024",2.00,"73,665,000,915"
6,Morgan Stanley,238.26M,"Dec 31, 2024",1.59,"58,505,760,549"
7,Price (T.Rowe) Associates Inc,220.11M,"Dec 31, 2024",1.47,"54,047,405,890"
8,Norges Bank Investment Management,187.16M,"Dec 31, 2024",1.25,"45,957,178,350"
9,JP Morgan Chase & Company,183.01M,"Dec 31, 2024",1.22,"44,937,652,036"


In [ ]:
def get_yahoo_economic_calendar(table_class: str = None) -> pd.DataFrame:
    """
    Extrat data from trading economics callendar.

    Parameters:
        classe_tabela (str): Class CSS from table to scrap (optional).

    Returns:
        pd.DataFrame: DataFrame with table content.
    """

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get("https://tradingeconomics.com/calendar", headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        if table_class:
            tabela = soup.find('table', {'class': table_class})
        else:
            tabela = soup.find('table')
        
        headers = [th.text.strip() for th in tabela.find_all('th')]
        print(headers)
        rows = []
        for row in tabela.find_all('tr')[1:]:
            cols = [td.text.strip() for td in row.find_all('td')]
            if cols: 
                rows.append(cols)

        df = pd.DataFrame(rows, columns=headers if headers else None)
        

    except requests.exceptions.RequestException as e:
        print(f"Error accessing URL: {e}")
    except Exception as e:
        print(f"Error processing data: {e}")

    return df


In [ ]:
def get_whalewisdom(ticker: str) -> pd.DataFrame:
    """
    Extracts data from WhaleWisdom for a given stock ticker.

    Parameters:
        ticker (str): Stock ticker symbol.

    Returns:
        pd.DataFrame: DataFrame with table content.
    """

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get(f"https://whalewisdom.com/stock/{ticker}", headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Encontrar a div com a classe "v-data-table__wrapper"
        div_wrapper = soup.find('div', {'class': 'v-window__container'})
       
        if not div_wrapper:
            raise ValueError("Div com a classe 'v-window__container' não encontrada.")

        # Buscar a primeira tabela dentro desta div
        tabela = div_wrapper.find('table')
        if not tabela:
            raise ValueError("Nenhuma tabela encontrada dentro da div especificada.")
        
        # Coletar os cabeçalhos da tabela ignorando ícones
        headers = [th.find('span').text.strip() for th in tabela.find_all('th') if th.find('span')]
        # print(headers)

        # Encontrando todas as linhas dentro da tabela (tbody > tr)
        rows = tabela.find("tbody").find_all("tr")
        print(rows)
        # Extraindo os dados de cada linha
        data = []
        for row in rows:
            columns = row.find_all("td")

            # Pegando os valores correspondentes
            institution_name = columns[0].text.strip()
            shares_held = columns[2].text.strip()
            portfolio_value = columns[3].text.strip()
            percentage_ownership = columns[4].text.strip()
            last_report_date = columns[-1].text.strip()

            data.append({
                "Instituição": institution_name,
                "Ações Detidas": shares_held,
                "Valor do Portfólio": portfolio_value,
                "Percentual de Participação": percentage_ownership,
                "Última Atualização": last_report_date
            })
        #print(data)
    
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL: {e}")
    except ValueError as e:
        print(f"Erro na extração de dados: {e}")
    except Exception as e:
        print(f"Erro inesperado: {e}")
    
    return pd.DataFrame()

In [52]:
df_whale_s = get_whalewisdom(ticker = "ctre")
df_whale_s

[<tr class="v-data-table__empty-wrapper"><td colspan="15">No data available</td></tr>]
Erro inesperado: list index out of range


""


In [67]:
base_url = 'https://whalewisdom.com/shell/command.html?args=%7B%22command%22:%22filer_lookup%22,%20%22name%22:%22berkshire%22%7D'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "application/json"
}

response = requests.get(base_url, headers=headers)
response.raise_for_status()

response.text

'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n    <title>Track Hedge Funds Using 13F Filings </title>\n    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no, minimal-ui">\n            <meta name="description" content="WhaleWisdom tracks 13F,  Schedule 13D, and 13G EDGAR filings by hedge funds. Hedge Fund Whale Backtesting and search tools "/>\n    <link rel="shortcut icon" type="image/x-icon" href="https://d27mjrcvcy56qq.cloudfront.net/images/favicon.ico"/>\n    <link rel="apple-touch-icon" sizes="57x57" href="https://d27mjrcvcy56qq.cloudfront.net/images/apple-touch-icon-57x57.png">\n    <link rel="apple-touch-icon" sizes="60x60" href="https://d27mjrcvcy56qq.cloudfront.net/images/apple-touch-icon-60x60.png">\n    <link rel="apple-touch-icon" sizes="72x72" href="https://d27mjrcvcy56qq.cloudfront.net/images/apple-touch-icon-72x72.png">\n    <link rel="apple-touch-icon" sizes="76x76" href="https://d27mjrcvcy56

In [25]:
import hashlib
import hmac
import base64
import json
import time
import urllib.parse
import requests

class WhaleWisdom:
    def __init__(self, secret_key, shared_key):
        self.secret_key = secret_key
        self.shared_key = shared_key
        self.timestamp = self.get_timestamp()

    def get_timestamp(self):
        """Gera o timestamp no formato UTC"""
        return time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())

    def signature(self, args):
        """Gera a assinatura HMAC-SHA1 em Base64"""
        message = f"{args}\n{self.timestamp}".encode('utf-8')
        hmac_digest = hmac.new(self.secret_key.encode('utf-8'), message, hashlib.sha1).digest()
        return base64.b64encode(hmac_digest).decode('utf-8')

    def encode(self, string):
        """Codifica a string para URL"""
        return urllib.parse.quote(string, safe='')

    def endpoint(self, args):
        """Constrói a URL da API do WhaleWisdom"""
        encoded_args = self.encode(json.dumps(args))
        api_sig = self.signature(json.dumps(args))
        encoded_timestamp = self.encode(self.timestamp)

        return f"https://whalewisdom.com/shell/command.json?args={encoded_args}&api_shared_key={self.shared_key}&api_sig={api_sig}&timestamp={encoded_timestamp}"

    def request(self, args):
        """Faz a requisição à API"""
        url = self.endpoint(args)
        response = requests.get(url)

        if response.status_code == 200:
            return response.text
        else:
            print(f"Erro {response.status_code}: {response.text}")
            return None


# **Exemplo de Uso**
WW_SHARED_KEY = "aTWxWrcILEOGqkAPZtAL"
WW_SECRET_KEY = "JGlAleUrGy1bWaLoiWwESNIbTHaovA0ocKOnr4MW"

ww = WhaleWisdom(secret_key=WW_SECRET_KEY, shared_key=WW_SHARED_KEY)

# Obter as participações (holdings) de um gestor específico
response = ww.request({
    "command": "stock_lookup",
    "symbol": "ctre"
})

data = json.loads(response)

stock_id = data["stocks"][0]["id"]
name = data["stocks"][0]["name"]
status = data["stocks"][0]["status"]
link = data["stocks"][0]["link"]

stock_id

167941

In [36]:
response = ww.request(
{"command":"holdings","stock_ids":[167941],"filer_ids":[373], "limit": 10, "all_quarters":0})
response


'{"errors":["Subscription limit has been reached for current period"]}'

In [19]:
response

'{"errors":["Subscription limit has been reached for current period"]}'

## STOCKS



In [1]:
import yfinance as yf
import pandas as pd
import talib
from django.http import JsonResponse
from datetime import datetime, timedelta


# Definir o ticker da ação
symbol = "AAPL"  # Apple Inc.

# Definir período e intervalo
period = "1y"       # Período de 1 mês
interval = "1d"      # Dados diários

In [ ]:
yahoo_symbol_info = yf.Ticker(symbol).info
yahoo_symbol_balancesheet = yf.Ticker(symbol).balance_sheet
yahoo_symbol_income = yf.Ticker(symbol).income_stmt

# total_equity
if 'Stockholders Equity' in yahoo_symbol_balancesheet.index:
    total_equity = yahoo_symbol_balancesheet.loc['Stockholders Equity'].iloc[0]
    if pd.isna(total_equity) or total_equity == 0:
        total_equity = "N/A"
else:
    total_equity = "N/A"
    
# cash_equivalents_short_term_investments
if 'Cash Cash Equivalents And Short Term Investments' in yahoo_symbol_balancesheet.index:
    cash_equivalents_short_term_investments = yahoo_symbol_balancesheet.loc['Cash Cash Equivalents And Short Term Investments'].iloc[0]
    if pd.isna(cash_equivalents_short_term_investments) or cash_equivalents_short_term_investments == 0:
        cash_equivalents_short_term_investments = "N/A"
else:
    cash_equivalents_short_term_investments = "N/A"

# cash_and_cash_equivalents
if 'Cash And Cash Equivalents' in yahoo_symbol_balancesheet.index:
    cash_and_cash_equivalents = yahoo_symbol_balancesheet.loc['Cash And Cash Equivalents'].iloc[0]
    if pd.isna(cash_and_cash_equivalents) or cash_and_cash_equivalents == 0:
        cash_and_cash_equivalents = "N/A"
else:
    cash_and_cash_equivalents = "N/A"

# current_liabilities
if 'Current Liabilities' in yahoo_symbol_balancesheet.index:
    current_liabilities = yahoo_symbol_balancesheet.loc['Current Liabilities'].iloc[0]
    if pd.isna(current_liabilities) or current_liabilities == 0:
        current_liabilities = "N/A"
else:
    current_liabilities = "N/A"

# cash_ratio
if cash_and_cash_equivalents != "N/A" and current_liabilities != "N/A" and current_liabilities != 0:
    cash_ratio = cash_and_cash_equivalents / current_liabilities
else:
    cash_ratio = "N/A"

# ebit
if 'EBIT' in yahoo_symbol_income.index:
    ebit = yahoo_symbol_income.loc["EBIT"].iloc[0]
    if pd.isna(ebit) or ebit == 0:
        ebit = "N/A"

# interest_expenses
if 'Interest Expense' in yahoo_symbol_income.index:
    interest_expenses = yahoo_symbol_income.loc["Interest Expense"].iloc[0]
    if pd.isna(interest_expenses) or interest_expenses == 0:
        interest_expenses = "N/A"

# interest_coverage_ratio
if ebit != "N/A" and interest_expenses not in ["N/A", 0]:
    interest_coverage_ratio = ebit / interest_expenses
else:
    interest_coverage_ratio = "N/A"

# total_assets
if 'Total Assets' in yahoo_symbol_balancesheet.index:
    total_assets = yahoo_symbol_balancesheet.loc['Total Assets'].iloc[0]
    if pd.isna(total_assets) or total_assets == 0:
        total_assets = "N/A"
else:
    total_assets = "N/A"

# debt_to_assets_ratio 
total_debt = yahoo_symbol_info.get("totalDebt", "N/A"),
if total_assets != "N/A" and total_debt != "N/A":
    debt_to_assets_ratio = total_debt / total_assets if total_assets != 0 else "N/A"
else:
    debt_to_assets_ratio = "N/A"

In [27]:
yahoo_symbol_income.loc["EBIT"].iloc[0]

123216000000.0

In [ ]:
yahoo_symbol_info = {
    # Liquidez e Solvência: Capacidade de pagamento
    "Quick Ratio": yahoo_symbol_info.get("quickRatio", "N/A"),
    "Current Ratio": yahoo_symbol_info.get("currentRatio", "N/A"),
    "Total Cash": yahoo_symbol_info.get("totalCash", "N/A"),
    "Total Debt": yahoo_symbol_info.get("totalDebt", "N/A"),
    "Total Equity": total_equity,
    # Aux
    "Cash Cash Equivalents And Short Term Investments": cash_equivalents_short_term_investments,
    "Cash And Cash Equivalents": cash_and_cash_equivalents,
    "Current Liabilities": current_liabilities,
    "EBIT": ebit,
    "Interes Expenses": interest_expenses,
    # Curto Prazo
    "Cash Ratio": cash_ratio,
    "Operating Cash Flow": yahoo_symbol_info.get("operatingCashflow", "N/A"),
    # Longo Prazo
    "Debt to Equity (D/E)": yahoo_symbol_info.get("debtToEquity", "N/A"),
    "Interest Coverage Ratio": interest_coverage_ratio,
    "Debt-to-Assets Ratio": debt_to_assets_ratio,

    # Lucratividade: Quão eficiente é a Apple em gerar lucro?
    "Margem Bruta": yahoo_symbol_info.get("grossMargins", "N/A"),
    "Margem Operacional": yahoo_symbol_info.get("operatingMargins", "N/A"),
    "Margem EBITDA": yahoo_symbol_info.get("ebitdaMargins", "N/A"),
    "Lucro Líquido (Net Income)": yahoo_symbol_info.get("netIncomeToCommon", "N/A"),
    "Profit Margin (Margem Líquida)": yahoo_symbol_info.get("profitMargins", "N/A"),
    "Retorno sobre Ativos (ROA)": yahoo_symbol_info.get("returnOnAssets", "N/A"),
    "Retorno sobre Patrimônio (ROE)": yahoo_symbol_info.get("returnOnEquity", "N/A"),

    # Crescimento: A Apple ainda cresce?
    "Crescimento de Receita (YoY)": yahoo_symbol_info.get("revenueGrowth", "N/A"),
    "Crescimento do Lucro Líquido (YoY)": yahoo_symbol_info.get("earningsQuarterlyGrowth", "N/A"),
    "Earnings Growth (Previsão de Crescimento de Lucros)": yahoo_symbol_info.get("earningsGrowth", "N/A"),
    
    # Valuation: A Apple está cara ou barata?
    "P/E Ratio (Preço/Lucro)": yahoo_symbol_info.get("trailingPE", "N/A"),
    "Forward P/E": yahoo_symbol_info.get("forwardPE", "N/A"),
    "PEG Ratio": yahoo_symbol_info.get("trailingPegRatio", "N/A"),
    "P/B Ratio (Preço/Valor Patrimonial)": yahoo_symbol_info.get("priceToBook", "N/A"),
    "EV/EBITDA": yahoo_symbol_info.get("enterpriseToEbitda", "N/A"),

    # Dividendos e Recompras de Ações
    "Dividend Yield": yahoo_symbol_info.get("dividendYield", "N/A"),
    "Payout Ratio": yahoo_symbol_info.get("payoutRatio", "N/A"),
    "Média de Dividend Yield (5 anos)": yahoo_symbol_info.get("fiveYearAvgDividendYield", "N/A"),

    # Risco e Sentimento de Mercado
    "Beta": yahoo_symbol_info.get("beta", "N/A"),
    "Risco de Auditoria": yahoo_symbol_info.get("auditRisk", "N/A"),
    "Risco do Conselho": yahoo_symbol_info.get("boardRisk", "N/A"),
    "Short Interest": yahoo_symbol_info.get("sharesPercentSharesOut", "N/A"),
    "Recomendação Média": yahoo_symbol_info.get("recommendationMean", "N/A"),
    "Preço-Alvo Médio": yahoo_symbol_info.get("targetMeanPrice", "N/A")
}


In [19]:
yahoo_symbol_info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p

In [ ]:
yahoo_symbol_balancesheet = yf.Ticker(symbol).balance_sheet
yahoo_symbol_balancesheet.index

Index(['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued',
       'Net Debt', 'Total Debt', 'Tangible Book Value', 'Invested Capital',
       'Working Capital', 'Net Tangible Assets', 'Capital Lease Obligations',
       'Common Stock Equity', 'Total Capitalization',
       'Total Equity Gross Minority Interest', 'Stockholders Equity',
       'Gains Losses Not Affecting Retained Earnings',
       'Other Equity Adjustments', 'Retained Earnings', 'Capital Stock',
       'Common Stock', 'Total Liabilities Net Minority Interest',
       'Total Non Current Liabilities Net Minority Interest',
       'Other Non Current Liabilities', 'Tradeand Other Payables Non Current',
       'Long Term Debt And Capital Lease Obligation',
       'Long Term Capital Lease Obligation', 'Long Term Debt',
       'Current Liabilities', 'Other Current Liabilities',
       'Current Deferred Liabilities', 'Current Deferred Revenue',
       'Current Debt And Capital Lease Obligation',
       'Current C

In [23]:
yahoo_symbol_cashflow = yf.Ticker(symbol).cash_flow
yahoo_symbol_cashflow.index

Index(['Free Cash Flow', 'Repurchase Of Capital Stock', 'Repayment Of Debt',
       'Issuance Of Debt', 'Issuance Of Capital Stock', 'Capital Expenditure',
       'Interest Paid Supplemental Data', 'Income Tax Paid Supplemental Data',
       'End Cash Position', 'Beginning Cash Position', 'Changes In Cash',
       'Financing Cash Flow', 'Cash Flow From Continuing Financing Activities',
       'Net Other Financing Charges', 'Cash Dividends Paid',
       'Common Stock Dividend Paid', 'Net Common Stock Issuance',
       'Common Stock Payments', 'Common Stock Issuance',
       'Net Issuance Payments Of Debt', 'Net Short Term Debt Issuance',
       'Net Long Term Debt Issuance', 'Long Term Debt Payments',
       'Long Term Debt Issuance', 'Investing Cash Flow',
       'Cash Flow From Continuing Investing Activities',
       'Net Other Investing Changes', 'Net Investment Purchase And Sale',
       'Sale Of Investment', 'Purchase Of Investment',
       'Net Business Purchase And Sale', 'Purch

In [24]:
yahoo_symbol_income = yf.Ticker(symbol).income_stmt
yahoo_symbol_income.index

Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Interest Income',
       'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Total Operating Income As Reported',
       'Diluted Average Shares', 'Basic Average Shares', 'Diluted EPS',
       'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Income Expense', 'Other Non Operating Income Expenses',
       'Net Non Operating Interest Income Expense',
       'Interest Expense Non Operating', 'Interest Income Non Operating',
       'Operating Inco

In [ ]:
https://medium.com/towards-data-science/pivot-points-calculation-in-python-for-day-trading-659c1e92d323


### ZONAS PIVOT! ###

## Data Collection Binance


In [ ]:
import requests

url = "https://api.binance.com/api/v3/ticker/price"
params = {"symbol": "BTCUSDT"}

response = requests.get(url, params=params)

if response.status_code == 200:
    symbol = response.json()["symbol"]
    price = response.json()["price"]
else:
    error = response.status_code

In [ ]:
from decimal import Decimal
from datetime import datetime


def get_crypto_symbol_24h(symbol : str):

    url = "https://api.binance.com/api/v3/ticker/24hr"
    params = {"symbol": symbol}

    response = requests.get(url, params=params)

    if response.status_code == 200:
        priceChangePercent = Decimal(response.json()["priceChangePercent"])
        weightedAvgPrice = Decimal(response.json()["weightedAvgPrice"])
        prevClosePrice = Decimal(response.json()["prevClosePrice"])
        priceChange = Decimal(response.json()["priceChange"])
        lastPrice = Decimal(response.json()["lastPrice"])
        lastQty = Decimal(response.json()["lastQty"])
        bidPrice = Decimal(response.json()["bidPrice"])
        bidQty = Decimal(response.json()["bidQty"])
        askPrice = Decimal(response.json()["askPrice"])
        askQty = Decimal(response.json()["askQty"])
        openPrice = Decimal(response.json()["openPrice"])
        highPrice = Decimal(response.json()["highPrice"])
        lowPrice = Decimal(response.json()["lowPrice"])
        volume = Decimal(response.json()["volume"])
        quoteVolume = Decimal(response.json()["quoteVolume"])
        openTime = datetime.fromtimestamp(response.json()["openTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
        closeTime = datetime.fromtimestamp(response.json()["closeTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
        firstId = response.json()["firstId"]
        lastId = response.json()["lastId"]
        count = response.json()["count"]
    else:
        print(f"Erro: {response.status_code} - {response.text}")

In [ ]:
url = "https://api.binance.com/api/v3/openOrders"
params = {"symbol": "BTCUSDT"}

response = requests.get(url, params=params)
response.json()


## TEST

In [ ]:
import yfinance as yf
from datetime import datetime
import warnings
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Suprimir avisos específicos
warnings.filterwarnings("ignore", category=FutureWarning, module="yfinance")


In [ ]:
def get_yahoo_stocks_top100_gainers(table_class: str = None) -> pd.DataFrame:
    """
    Extrat data from yahoo top 100 gainers

    Parameters:
        classe_tabela (str): Class CSS from table to scrap (optional).

    Returns:
        pd.DataFrame: DataFrame with table content.
    """

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get("https://finance.yahoo.com/markets/stocks/gainers/?start=0&count=100", headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        if table_class:
            tabela = soup.find('table', {'class': table_class})
        else:
            tabela = soup.find('table')

        headers = [th.text.strip() for th in tabela.find_all('th')]

        rows = []
        for row in tabela.find_all('tr')[1:]:
            cols = [td.text.strip() for td in row.find_all('td')]
            if cols: 
                rows.append(cols)

        df = pd.DataFrame(rows, columns=headers if headers else None)
        df["Price"] = df["Price"].str.extract(r"^([\d\.]+)")
        df["Change"] = df["Change"].str.replace("+", "", regex=False)
        df["Change %"] = df["Change %"].str.replace("%", "", regex=False)
        df["Volume"] = df["Volume"].str.replace("-", "0", regex=False)
        df["Avg Vol (3M)"] = df["Avg Vol (3M)"].str.replace("-", "0", regex=False)
        df["Market Cap"] = df["Market Cap"].str.replace("-", "0", regex=False)
        df["P/E Ratio (TTM)"] = df["P/E Ratio (TTM)"].str.replace("-", "0", regex=False)
        df["52 Wk Change %"] = df["52 Wk Change %"].str.replace("%", "", regex=False)
        df.drop(columns=["52 Wk Range"], inplace=True)
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error accessing URL: {e}")
    except Exception as e:
        print(f"Error processing data: {e}")


In [ ]:
df = get_yahoo_stocks_top100_gainers()
df

,Symbol,Name,,Price,Change,Change %,Volume,Avg Vol (3M),Market Cap,P/E Ratio (TTM),52 Wk Change %
0,PONY,Pony AI Inc.,,23.61,5.73,+32.05,6.103M,1.288M,8.239B,0,96.75
1,CELH,"Celsius Holdings, Inc.",,32.62,7.09,+27.77,61.72M,9.391M,7.666B,72.49,-50.24
2,VICR,Vicor Corporation,,63.67,11.81,+22.77,1.277M,"239,154",2.874B,454.79,66.24
3,TME,Tencent Music Entertainment Group,,14.38,1.55,+12.08,23.381M,7.168M,22.618B,27.65,38.00
4,TLX,Telix Pharmaceuticals Limited,,19.50,2.09,+12.00,"39,652","15,322",6.527B,162.50,28.71
...,...,...,...,...,...,...,...,...,...,...,...
57,ESGRP,Enstar Group Limited,,20.87,0.64,+3.16,"16,904","58,144",2.937B,0,-18.38
58,AWK,"American Water Works Company, Inc.",,132.68,4.01,+3.12,2.117M,1.262M,25.866B,24.62,12.99
59,BF-A,Brown-Forman Corporation,,32.28,0.96,+3.07,"119,446","143,167",15.191B,15.23,-44.56
60,GIS,"General Mills, Inc.",,61.00,1.81,+3.06,5.218M,4.504M,33.625B,13.26,-6.15
